In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import random
import re

class AllStudiosScraper:
    BASE_URL = "https://www.allstudios.co.uk"
    
    def __init__(self, output_file='data/studios_data.csv'):
        self.output_file = output_file
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
    
    def get_studio_list(self):
        """Retrieve list of studios from the main page"""
        url = f"{self.BASE_URL}/studios/recording-studio?clear=true"
        
        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            
            studios = []
            for li in soup.select('ul.studios-list li'):
                studio_link = li.find('a', href=lambda href: href and '/studios/recording-studio/' in href)
                studio_website = li.find('a', class_='studio_index_button1')
                
                if studio_link and studio_website:
                    studio_name = studio_link.text.strip()
                    studio_page = self.BASE_URL + studio_link['href']
                    website = studio_website.get('href', 'N/A')
                    
                    studios.append({
                        'name': studio_name,
                        'page': studio_page,
                        'website': website
                    })
            
            return studios
        
        except requests.RequestException as e:
            print(f"Error fetching studio list: {e}")
            return []
###    
    def extract_location_from_map(self, soup):
        """Extract location information from Google Maps static map URL"""
        location_info = {
            'latitude': 'N/A',
            'longitude': 'N/A',
            'city': 'N/A',
            'country': 'N/A'
        }

        # Find the Google Maps static map image
        map_img = soup.find('img', src=re.compile(r'https://maps\.googleapis\.com/maps/api/staticmap'))

        if map_img and map_img.get('src'):
            map_url = map_img['src']

            # Extract coordinates
            coord_match = re.search(r'center=([^&]+)', map_url)
            if coord_match:
                try:
                    latitude, longitude = coord_match.group(1).split(',')
                    location_info['latitude'] = latitude.strip()
                    location_info['longitude'] = longitude.strip()

                except ValueError:
                    print("Could not parse coordinates from map URL")

        return location_info
###   
    def extract_contact_info(self, soup):
        """Advanced contact information extraction"""
        contact_info = {
            'postal_address': 'N/A',
            'contact_name': 'N/A',
            'phone': 'N/A',
            'email': 'N/A'
        }

        # Strategy 1: Look for specific table with studio-view class
        studio_table = soup.find('table', class_='studio-view')
        if studio_table:
            # Try extracting from table rows
            rows = studio_table.find_all('tr')
            for row in rows:
                row_text = row.get_text(strip=True)

                # Extract contact name
                if 'Contact Name:' in row_text:
                    contact_info['contact_name'] = row_text.replace('Contact Name:', '').strip()

                # Method 1: Simple string slicing for phone number
                if row_text.startswith('Tel:'):
                    contact_info['phone'] = row_text[5:].strip()

                # Method 2: Regex extraction for phone number
                phone_match = re.search(r'Tel:\s*(.+)', row_text)
                if phone_match and contact_info['phone'] == 'N/A':
                    contact_info['phone'] = phone_match.group(1).strip()

        # Strategy 2: Look for contact information in div elements (kept from original code)
        contact_divs = soup.find_all(['div', 'p'], text=re.compile(r'(Contact Name|Tel:|Phone:|Telephone:)'))
        for div in contact_divs:
            div_text = div.get_text(strip=True)

            # Extract contact name
            name_match = re.search(r'Contact Name:\s*(.+)', div_text, re.IGNORECASE)
            if name_match:
                contact_info['contact_name'] = name_match.group(1).strip()

            # Extract phone number
            phone_match = re.search(r'(Tel:|Phone:|Telephone:)\s*(\+?[\d\s\(\)-]+)', div_text, re.IGNORECASE)
            if phone_match and contact_info['phone'] == 'N/A':
                contact_info['phone'] = phone_match.group(2).strip()

        # Strategy 3: Look for email link
        email_link = soup.find('a', href=re.compile(r'^mailto:'))
        if email_link:
            contact_info['email'] = email_link['href'].replace('mailto:', '').strip()

        # Strategy 4: Look for postal address
        if studio_table:
            address_row = studio_table.find('p')
            if address_row:
                contact_info['postal_address'] = address_row.get_text(strip=True)

        return contact_info
    
    def get_studio_details(self, studio_page):
        """Retrieve detailed information for a specific studio"""
        try:
            response = requests.get(studio_page, headers=self.headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Use advanced extraction method
            return self.extract_contact_info(soup)
        
        except requests.RequestException as e:
            print(f"Error fetching studio details for {studio_page}: {e}")
            return {
                'postal_address': 'N/A',
                'contact_name': 'N/A',
                'phone': 'N/A',
                'email': 'N/A'
            }
    
    def scrape_all_studios(self, max_studios=None):
        """Scrape details for studios"""
        studios = self.get_studio_list()
        
        # Limit studios if max_studios is specified
        if max_studios is not None:
            studios = studios[:max_studios]
        
        # Prepare CSV file
        with open(self.output_file, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['Studio Name', 'Studio Website', 'Contact Name', 'Email', 'Phone', 'Postal Address']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            
            # Track progress
            total_studios = len(studios)
            print(f"Total studios to scrape: {total_studios}")
            
            for index, studio in enumerate(studios, 1):
                print(f"Scraping details for {studio['name']} (Studio {index} of {total_studios})...")
                
                # Get additional details
                details = self.get_studio_details(studio['page'])
                
                # Combine studio info with details
                full_details = {
                    'Studio Name': studio['name'],
                    'Studio Website': studio['website'],
                    'Contact Name': details.get('contact_name', 'N/A'),
                    'Email': details.get('email', 'N/A'),
                    'Phone': details.get('phone', 'N/A'),
                    'Postal Address': details.get('postal_address', 'N/A')
                }
                
                writer.writerow(full_details)
                
                # Random delay to be respectful of the website
                time.sleep(random.uniform(1, 3))
        
        print(f"Scraping complete. Data saved to {self.output_file}")

# Example usage
if __name__ == "__main__":
    scraper = AllStudiosScraper()
    # Scrape first 10 studios for testing
    scraper.scrape_all_studios(max_studios=5)

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import random
import re

class AllStudiosScraper:
    BASE_URL = "https://www.allstudios.co.uk"
    
    def __init__(self, output_file='data/studios_data.csv'):
        self.output_file = output_file
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
    
    def get_studio_list(self):
        """Retrieve list of studios from the main page"""
        url = f"{self.BASE_URL}/studios/recording-studio?clear=true"
        
        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            
            studios = []
            for li in soup.select('ul.studios-list li'):
                studio_link = li.find('a', href=lambda href: href and '/studios/recording-studio/' in href)
                studio_website = li.find('a', class_='studio_index_button1')
                
                if studio_link and studio_website:
                    studio_name = studio_link.text.strip()
                    studio_page = self.BASE_URL + studio_link['href']
                    website = studio_website.get('href', 'N/A')
                    
                    studios.append({
                        'name': studio_name,
                        'page': studio_page,
                        'website': website
                    })
            
            return studios
        
        except requests.RequestException as e:
            print(f"Error fetching studio list: {e}")
            return []
###    
    def extract_location_from_map(self, soup):
        """Extract location information from Google Maps static map URL"""
        location_info = {
            'latitude': 'N/A',
            'longitude': 'N/A',
            'city': 'N/A',
            'country': 'N/A'
        }

        # Find the Google Maps static map image
        map_img = soup.find('img', src=re.compile(r'https://maps\.googleapis\.com/maps/api/staticmap'))

        if map_img and map_img.get('src'):
            map_url = map_img['src']

            # Extract coordinates
            coord_match = re.search(r'center=([^&]+)', map_url)
            if coord_match:
                try:
                    latitude, longitude = coord_match.group(1).split(',')
                    location_info['latitude'] = latitude.strip()
                    location_info['longitude'] = longitude.strip()

                except ValueError:
                    print("Could not parse coordinates from map URL")

        return location_info
###   
    def extract_contact_info(self, soup):
        """Advanced contact information extraction"""
        contact_info = {
            'postal_address': 'N/A',
            'contact_name': 'N/A',
            'phone': 'N/A',
            'email': 'N/A'
        }

        # Strategy 1: Look for specific table with studio-view class
        studio_table = soup.find('table', class_='studio-view')
        if studio_table:
            # Try extracting from table rows
            rows = studio_table.find_all('tr')
            for row in rows:
                row_text = row.get_text(strip=True)

                # Extract contact name
                if 'Contact Name:' in row_text:
                    contact_info['contact_name'] = row_text.replace('Contact Name:', '').strip()

                # Method 1: Simple string slicing for phone number
                if row_text.startswith('Tel:'):
                    contact_info['phone'] = row_text[5:].strip()

                # Method 2: Regex extraction for phone number
                phone_match = re.search(r'Tel:\s*(.+)', row_text)
                if phone_match and contact_info['phone'] == 'N/A':
                    contact_info['phone'] = phone_match.group(1).strip()

        # Strategy 2: Look for contact information in div elements (kept from original code)
        contact_divs = soup.find_all(['div', 'p'], text=re.compile(r'(Contact Name|Tel:|Phone:|Telephone:)'))
        for div in contact_divs:
            div_text = div.get_text(strip=True)

            # Extract contact name
            name_match = re.search(r'Contact Name:\s*(.+)', div_text, re.IGNORECASE)
            if name_match:
                contact_info['contact_name'] = name_match.group(1).strip()

            # Extract phone number
            phone_match = re.search(r'(Tel:|Phone:|Telephone:)\s*(\+?[\d\s\(\)-]+)', div_text, re.IGNORECASE)
            if phone_match and contact_info['phone'] == 'N/A':
                contact_info['phone'] = phone_match.group(2).strip()

        # Strategy 3: Look for email link
        email_link = soup.find('a', href=re.compile(r'^mailto:'))
        if email_link:
            contact_info['email'] = email_link['href'].replace('mailto:', '').strip()

        # Strategy 4: Look for postal address
        if studio_table:
            address_row = studio_table.find('p')
            if address_row:
                contact_info['postal_address'] = address_row.get_text(strip=True)

        return contact_info
    
    def get_studio_details(self, studio_page):
        """Retrieve detailed information for a specific studio"""
        try:
            response = requests.get(studio_page, headers=self.headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # Use advanced extraction methods
            contact_info = self.extract_contact_info(soup)
            location_info = self.extract_location_from_map(soup)

            # Combine the dictionaries
            full_details = {**contact_info, **location_info}
            return full_details

        except requests.RequestException as e:
            print(f"Error fetching studio details for {studio_page}: {e}")
            return {
                'postal_address': 'N/A',
                'contact_name': 'N/A',
                'phone': 'N/A',
                'email': 'N/A',
                'latitude': 'N/A',
                'longitude': 'N/A',
                'city': 'N/A',
                'country': 'N/A'
            }
    
    def scrape_all_studios(self, max_studios=None):
        """Scrape details for studios"""
        studios = self.get_studio_list()
        
        # Limit studios if max_studios is specified
        if max_studios is not None:
            studios = studios[:max_studios]

        # Prepare CSV file
        with open(self.output_file, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['Studio Name', 'Studio Website', 'Contact Name', 'Email', 'Phone', 
                          'Postal Address', 'Latitude', 'Longitude', 'City', 'Country']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

                 # Track progress
            total_studios = len(studios)
            print(f"Total studios to scrape: {total_studios}")
            
            for index, studio in enumerate(studios, 1):
                print(f"Scraping details for {studio['name']} (Studio {index} of {total_studios})...")
                
                # Get additional details
                details = self.get_studio_details(studio['page'])

                # Combine studio info with details

                full_details = {
                    'Studio Name': studio['name'],
                    'Studio Website': studio['website'],
                    'Contact Name': details.get('contact_name', 'N/A'),
                    'Email': details.get('email', 'N/A'),
                    'Phone': details.get('phone', 'N/A'),
                    'Postal Address': details.get('postal_address', 'N/A'),
                    'Latitude': details.get('latitude', 'N/A'),
                    'Longitude': details.get('longitude', 'N/A'),
                    'City': details.get('city', 'N/A'),
                    'Country': details.get('country', 'N/A')
                }

                writer.writerow(full_details)

                # Random delay to be respectful of the website
                time.sleep(random.uniform(1, 3))
        
        print(f"Scraping complete. Data saved to {self.output_file}")

# Example usage
if __name__ == "__main__":
    scraper = AllStudiosScraper()
    # Scrape first 10 studios for testing
    scraper.scrape_all_studios()

Total studios to scrape: 481
Scraping details for Far Heath Studios (Studio 1 of 481)...


/var/folders/zf/w4lxv8m15z9442k97fpp97ch0000gn/T/ipykernel_15660/3875694485.py:107: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  contact_divs = soup.find_all(['div', 'p'], text=re.compile(r'(Contact Name|Tel:|Phone:|Telephone:)'))


Scraping details for 123 Studios (Studio 2 of 481)...
Scraping details for 3Sixty Studios (Studio 3 of 481)...
Scraping details for 5A Studios (Studio 4 of 481)...
Scraping details for 7 West Studios (Studio 5 of 481)...
Scraping details for 77 Sound Studio (Studio 6 of 481)...
Scraping details for 80 HERTZ Recording & Post Production Studios (Studio 7 of 481)...
Scraping details for A Bloody Good Record Recording Studio (Studio 8 of 481)...
Scraping details for A Sharp Recording Studios (Studio 9 of 481)...
Scraping details for A-Tonal Recording Studio (Studio 10 of 481)...
Scraping details for A.K.A Studio (Studio 11 of 481)...
Scraping details for Abbey Music Studios (Studio 12 of 481)...
Scraping details for Abbey Road Penthouse (Studio 13 of 481)...
Scraping details for Abbey Road Studio 1 (Studio 14 of 481)...
Scraping details for Abbey Road Studio 2 (Studio 15 of 481)...
Scraping details for Abbey Road Studio 3 (Studio 16 of 481)...
Scraping details for Absolute Music Studios (S

Scraping details for Fairview Recording Studio (Studio 124 of 481)...
Scraping details for Fantasy Recording Studios: Studio A (Studio 125 of 481)...
Scraping details for Fantasy Recording Studios: Studio D (Studio 126 of 481)...
Scraping details for  (Studio 127 of 481)...
Scraping details for Faultline Studios (Studio 128 of 481)...
Scraping details for Finnvox Studios (Studio 129 of 481)...
Scraping details for Fire K Studios (Studio 130 of 481)...
Scraping details for Fireplace Studios (Studio 131 of 481)...
Scraping details for Flank Recording Studio (Studio 132 of 481)...
Scraping details for Flipside Recording Studios (Studio 133 of 481)...
Scraping details for Fluidity-Studios (Studio 134 of 481)...
Scraping details for Fluke Productions (Studio 135 of 481)...
Scraping details for Flux Studios: Dangerous Room (Studio 136 of 481)...
Scraping details for Flux Studios: Fabulous Room (Studio 137 of 481)...
Scraping details for Flux Studios: Revolution Room (Studio 138 of 481)...
Sc

Scraping details for Mwnci (Monkey) Studios (Studio 251 of 481)...
Scraping details for Nevo Sound Studios (Studio 252 of 481)...
Scraping details for NextDoorStudio (Studio 253 of 481)...
Scraping details for NightBird Recording Studio A (Studio 254 of 481)...
Scraping details for NightBird Recording Studio B (Studio 255 of 481)...
Scraping details for No Shame Labs (Studio 256 of 481)...
Scraping details for North Seven Studios (Studio 257 of 481)...
Scraping details for Oak Recording Studios (Studio 258 of 481)...
Scraping details for Old Foundry Studios (Studio 259 of 481)...
Scraping details for One Louder Studio 1 (Studio 260 of 481)...
Scraping details for One Louder Studio 2 (Studio 261 of 481)...
Scraping details for Online Music Mixing (Studio 262 of 481)...
Scraping details for Orchard Recording Studios (Studio 263 of 481)...
Scraping details for Outhouse Studios (Studio 264 of 481)...
Scraping details for Oxford Audio Post Production (Studio 265 of 481)...
Scraping details 

Scraping details for Squirky Studios (Studio 376 of 481)...
Scraping details for SSR London Ghost Studio (Studio 377 of 481)...
Scraping details for SSR London Icon Studio (Studio 378 of 481)...
Scraping details for SSR London Neve Studio (Studio 379 of 481)...
Scraping details for Stag Studios (Studio 380 of 481)...
Scraping details for Start Together: Studio 1 (Studio 381 of 481)...
Scraping details for Start Together: Studio 2 (Studio 382 of 481)...
Scraping details for Start Together: Studio 3 (Studio 383 of 481)...
Scraping details for State Of The Ark Studios - CURRENTLY LONGTERM LET (Studio 384 of 481)...
Scraping details for Steelworks Studio 1 (Studio 385 of 481)...
Scraping details for Strange Weather (Studio 386 of 481)...
Scraping details for Strongroom Studio 1 (Studio 387 of 481)...
Scraping details for Strongroom Studio 3 (Studio 388 of 481)...
Scraping details for Strongroom Studio 4 (Studio 389 of 481)...
Scraping details for Strummers Recording Studio (Studio 390 of 4

In [6]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

# Load the CSV file
file_path = 'data/studios_data.csv'
df = pd.read_csv(file_path)

# Initialize Geolocator
geolocator = Nominatim(user_agent="geoapi")

# Function to validate and reverse geocode coordinates
def get_location(lat, lon):
    try:
        # Check for valid coordinates
        if pd.isna(lat) or pd.isna(lon):
            return "Invalid", "Invalid"
        if not (-90 <= lat <= 90) or not (-180 <= lon <= 180):
            return "Invalid", "Invalid"
        
        # Perform reverse geocoding
        location = geolocator.reverse((lat, lon), timeout=10)
        if location and location.raw.get('address'):
            address = location.raw['address']
            city = address.get('city', address.get('town', address.get('village', 'Unknown')))
            country = address.get('country', 'Unknown')
            return city, country
        return "Unknown", "Unknown"
    except GeocoderTimedOut:
        return "Timeout", "Timeout"
    except Exception as e:
        return "Error", "Error"

# Apply the function only to rows with valid latitude and longitude
df['City'], df['Country'] = zip(*df.apply(
    lambda row: get_location(row.get('Latitude'), row.get('Longitude')), axis=1
))

# Save the updated DataFrame
output_file = 'data/studios_contact_info.csv'
df.to_csv(output_file, index=False)

print(f"Updated data saved to {output_file}")


Updated data saved to studios_contact_info.csv


In [12]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time

def is_valid_coordinate(value):
    """
    Check if the value is a valid numeric coordinate
    """
    try:
        # Convert to float and check if it's a valid number
        coord = float(value)
        # Check if coordinate is within plausible range
        # Latitude: -90 to 90, Longitude: -180 to 180
        return -90 <= coord <= 90 or -180 <= coord <= 180
    except (TypeError, ValueError):
        return False

def extract_location_from_coordinates(row, geolocator):
    """
    Use geopy to extract city and country from coordinates
    """
    # Check if both longitude and latitude are valid
    if (is_valid_coordinate(row['Longitude']) and 
        is_valid_coordinate(row['Latitude'])):
        try:
            # Add a small delay to avoid overwhelming the geocoding service
            time.sleep(1)
            
            # Reverse geocode
            location = geolocator.reverse(f"{row['Latitude']}, {row['Longitude']}")
            
            if location:
                # Extract address components
                address = location.raw.get('address', {})
                
                # Try to extract city (with multiple fallback options)
                city = (
                    address.get('city') or 
                    address.get('town') or 
                    address.get('village') or 
                    address.get('municipality') or 
                    'N/A'
                )
                
                # Extract country
                country = address.get('country', 'N/A')
                
                return pd.Series({'City': city, 'Country': country})
            
        except (GeocoderTimedOut, GeocoderServiceError) as e:
            print(f"Geocoding error for coordinates {row['Latitude']}, {row['Longitude']}: {e}")
    
    # Return N/A if no valid location found
    return pd.Series({'City': 'N/A', 'Country': 'N/A'})

def clean_coordinates_and_extract_location(df):
    """
    Clean Longitude and Latitude columns and extract location
    """
    # Create a copy of the dataframe to avoid SettingWithCopyWarning
    cleaned_df = df.copy()
    
    # Initialize geolocator
    geolocator = Nominatim(user_agent="studio_location_extractor")
    
    # Check if both Longitude and Latitude are valid
    cleaned_df['Valid_Coordinates'] = (
        cleaned_df['Longitude'].apply(is_valid_coordinate) & 
        cleaned_df['Latitude'].apply(is_valid_coordinate)
    )
    
    # For rows without valid coordinates, set to NaN
    mask = ~cleaned_df['Valid_Coordinates']
    cleaned_df.loc[mask, 'Longitude'] = np.nan
    cleaned_df.loc[mask, 'Latitude'] = np.nan
    
    # Extract city and country from valid coordinates
    location_info = cleaned_df.apply(
        lambda row: extract_location_from_coordinates(row, geolocator), 
        axis=1
    )
    
    # Update City and Country columns for valid coordinates
    valid_coords_mask = cleaned_df['Valid_Coordinates']
    cleaned_df.loc[valid_coords_mask, 'City'] = location_info.loc[valid_coords_mask, 'City']
    cleaned_df.loc[valid_coords_mask, 'Country'] = location_info.loc[valid_coords_mask, 'Country']
    
    # Drop the temporary validation column
    cleaned_df = cleaned_df.drop(columns=['Valid_Coordinates'])
    
    return cleaned_df

# Read the CSV file
df = pd.read_csv('data/studios_data.csv')

# Clean the dataframe
cleaned_df = clean_coordinates_and_extract_location(df)

# Save the cleaned dataframe
cleaned_df.to_csv('data/studios_data_cleaned.csv', index=False)

print(f"Updated data saved")

Updated data saved


/var/folders/zf/w4lxv8m15z9442k97fpp97ch0000gn/T/ipykernel_15660/507719236.py:87: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Coton' 'London' 'London' 'City of Westminster' 'Glasgow' 'Dulliken'
 'Manchester' 'City of New York' 'Sydney' 'London' 'Rīga' 'London'
 'London' 'London' 'London' 'London' 'Bournemouth'
 'Marrakech ⵎⵕⵕⴰⴽⵯⵛ مراكش' 'London' 'London' 'London' 'London' 'London'
 'London' 'Manchester' 'Manchester' 'London' 'Chelmsford' 'Verteillac'
 'München' 'Runcorn' 'Gravesham' 'Test Valley' 'Plymouth' 'London'
 'London' 'Pezzeit' 'London' 'London' 'Heywood' 'Bristol' 'McCandless'
 'Wealden' 'Brighton' 'Calderdale' 'Geisenfeld' 'City of New York'
 'City of New York' 'City of New York' 'City of New York'
 'City of New York' 'City of New York' 'City of New York'
 'Frankfurt am Main' 'Leeds' 'Newcastle upon Tyne' 'Bispham'
 'Forest of Dean' 'London' 'Newcastle upon Tyne' 'London' 'London'
 'London' 'Rugby' 'M

In [ ]:
# Display summary of changes
print("Original DataFrame:")

In [ ]:
print(df[['Longitude', 'Latitude', 'City', 'Country']].isna().sum())

In [ ]:
print("\nCleaned DataFrame:")

In [ ]:
print(cleaned_df[['Longitude', 'Latitude', 'City', 'Country']].isna().sum())

In [ ]:
# Display a few rows to verify
print("\nSample Rows:")

In [10]:
print(cleaned_df[['Studio Name', 'Longitude', 'Latitude', 'City', 'Country']].head())

Updated data saved to studios_contact_info.csv
